In [1]:
# Python notebook to:
# 1. Download model files from Hugging Face and store temporarily on node
# 2. Upload model files to COS
# 3. Utilites to clean up the temporarily stored model files
#
#
# ***** Make sure the workbench is configured with at least 60GB storage.
# ***** Make sure the current working directory has at least 40GB available disk space.
# ***** Make sure to clean up the downloaded files on the the pod.
#TBD Create a pipeline https://www.kubeflow.org/docs/components/pipelines/user-guides/core-functions/compile-a-pipeline/

In [2]:
!pip install boto3==1.35.55 
!pip install botocore==1.35.55
!pip install huggingface_hub


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import shutil
import os
import boto3
import botocore
import string
import random

In [3]:
# CONFIGURATION

# Model to download from Hugging Face
# Get from https://huggingface.co/models
# E.g https://huggingface.co/ibm-granite/granite-3.3-8b-instruct
#download_model_id='ibm-granite/granite-3.3-8b-instruct'

download_model_id='meta-llama/Llama-3.2-3B-Instruct'
#https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct/tree/main

# COS bucket and credentials for upload
access_key_id = "0bceecea1e564a9aaf26cc67d27dd410" 
secret_access_key = "43f154c27aef623c39da5ffcdc857e517e5eabdcfcc522c5" 
endpoint_url = "https://s3.br-sao.cloud-object-storage.appdomain.cloud"
region_name = "br-sao"


In [16]:
# Check current directory storage and contents
# Get the current directory path
current_directory = os.getcwd()  # Replace with the actual directory path
#current_directory = '.'  # Current working directory

current_directory_contents = os.listdir('.')
# Print curent directoy contents
print(f"Contents of current directory: {current_directory_contents}")

# Get disk usage statistics
total, used, free = shutil.disk_usage(current_directory)

# Convert bytes to a more readable unit (e.g., GB)
total_gb = total / (1024**3)
used_gb = used / (1024**3)
free_gb = free / (1024**3)

# Print disk storage space
print(f"Current directory:{current_directory}")
print(f"Disk Usage for {current_directory}:")
print(f"Total: {total_gb:.2f} GB")
print(f"Used: {used_gb:.2f} GB")
print(f"Free: {free_gb:.2f} GB")

print(f"***********************************")
print(f"**** BEFORE PROCEEDING FURTHER ****")
print(f"Make sure the current working directory has at least 39GB available Free disk space.")
print(f"This space is needed to download the model repository/files from Hugging Face.")
print(f"***********************************")

Contents of current directory: ['.cache', 'Untitled1.ipynb', 'test_model_download_pipeline.yaml', '.npm', 'test_model_all_download_pipeline.yaml', 'NB-pipeline-dev-helper.ipynb', 'NB-ai-agent-v2-loan-risk-for-rhoai.ipynb', '.ipynb_checkpoints', 'gs_model_pipeline.yaml', 'mcp3-ai-agent-bank-loan.ipynb', '.virtual_documents', '.config', 'writing_a_test_file.txt', 'lost+found', 'mcp3-wxai-ai-agent-bank-loan.ipynb', 'Untitled.ipynb', 'gs_hf_aws_pipeline.ipynb', 'file-write-pipeline.yaml', 'full_modeld_pipeline.yaml', '.ipython', '.local', 'NB-gen-rhoai-model-setup-pipeline.ipynb', '.locks', 'full_model_pipeline.yaml', 'my_output_model_path', '.jupyter', 'NB-get-and-upload-model.ipynb']
Current directory:/opt/app-root/src
Disk Usage for /opt/app-root/src:
Total: 39.20 GB
Used: 0.03 GB
Free: 39.15 GB
***********************************
**** BEFORE PROCEEDING FURTHER ****
Make sure the current working directory has at least 39GB available Free disk space.
This space is needed to download the 

In [17]:
# Confirm free disk storage space above before proceeding

In [19]:
# Download the model from Hugging Face 
# This step will take a few minutes.

from huggingface_hub import snapshot_download
repo_path = snapshot_download(
    #use_auth_token="hf_dXXXXXXXXXXXXXXXXX", # hf auth required for some models
    repo_id=download_model_id,
    repo_type="model",
    cache_dir="."
)
print(f"Repository downloaded to: {repo_path}")

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

USE_POLICY.md:   0%|          | 0.00/6.02k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

original/consolidated.00.pth:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

Repository downloaded to: ./models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95


In [20]:
local_models_directory=repo_path
print(f"Repository downloaded to local_models_directory: {local_models_directory}")

Repository downloaded to local_models_directory: ./models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95


In [21]:
print("Contents of model directory:")
local_models_directory_contents = os.listdir(local_models_directory)
print(local_models_directory_contents)

Contents of model directory:
['README.md', 'model-00002-of-00002.safetensors', 'tokenizer.json', 'generation_config.json', 'model.safetensors.index.json', 'config.json', 'USE_POLICY.md', '.gitattributes', 'original', 'tokenizer_config.json', 'special_tokens_map.json', 'model-00001-of-00002.safetensors', 'LICENSE.txt']


In [22]:
# Upload model files to IBM Cloud COS instance bucket

In [23]:
# Create an S3 resource object with credentials
s3_resource = boto3.resource(
    "s3",
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
    region_name=region_name,
    endpoint_url=endpoint_url
)

In [24]:
# Create the bucket
# Model bucket name is generated from the model name. No configuration needed
# e.g., granite-3.3-8b-instruct-<suffix> , 'granite-3.3-8b-instruct-HJ8mKd'
model_files_bucket_name = (download_model_id.split('/')[1]+'-'+''.join(random.choices(string.ascii_letters + string.digits, k=5))).lower()

model_files_bucket = s3_resource.create_bucket(
    ACL='private',
    Bucket=model_files_bucket_name,
    # By default bucket of type standard is created.
    # To create bucket of type smart tier, uncomment code below
    # Refer: https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-classes#classes
    #CreateBucketConfiguration={
    #    'LocationConstraint': region_name+'-smart' 
    #}
)

#print(model_files_bucket)
print(f"Created bucket for the model files: {model_files_bucket_name}")

Created bucket for the model files: llama-3.2-3b-instruct-wglo4


In [25]:
# Upload the model files to the bucket
# function definitions
def list_objects(bucket, prefix):
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        print(obj.key)
    #To list all files use: list_objects(model_files_bucket'')
    #To list all files with a prefix eg model-files use: list_objects(model_files_bucket,'model-files')
    #To change to a different bucket
    #bucket_to_list=s3_resource.Bucket("granite-3.3-8b-instruct-v0711")

        
def upload_directory_to_s3(local_directory, s3_prefix):
    num_files = 0
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            model_files_bucket.upload_file(file_path, s3_key)
            num_files += 1
    return num_files


In [26]:
# Upload the model files to the bucket
# This steps can take a few minutes.

if not os.path.isdir(local_models_directory):
    raise ValueError(f"The directory '{local_models_directory}' does not exist."
                     "Check if model was correctly downloaded in previous steps.")
else:

    print(f"*** Starting to upload model files to bucket: {model_files_bucket_name} ***")
    print(f"This step will take a few minutes. Progress updates below as each file gets uploaded.")
    model_files_prefix="model-files"
    num_files = upload_directory_to_s3(local_models_directory, model_files_prefix)
    # NOTE: "model-files" is a prefix added to each file that is uploaded. A path is required for configuration in RHOAI.
    print(f"*** Completed uploading model files to bucket: {model_files_bucket_name} ***")
    if num_files == 0:
        raise ValueError("No files uploaded. ")
    else:
        print(f" ")
        print(f"Saved {num_files} model files to the bucket \"{model_files_bucket_name}\"")
        print(f"Each file has path prefix \"{model_files_prefix}\". This path will be required during model deployment configuration")
        print(f" ")
        print(f"Here is the list of files uploaded to IBM Cloud COS bucket \"{model_files_bucket_name}\"")
        print(list_objects(model_files_bucket, 'model-files'))

*** Starting to upload model files to bucket: llama-3.2-3b-instruct-wglo4 ***
This step will take a few minutes. Progress updates below as each file gets uploaded.
./models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/README.md -> model-files/README.md
./models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/model-00002-of-00002.safetensors -> model-files/model-00002-of-00002.safetensors
./models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/tokenizer.json -> model-files/tokenizer.json
./models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/generation_config.json -> model-files/generation_config.json
./models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95/model.safetensors.index.json -> model-files/model.safetensors.index.json
./models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b

In [27]:
# **** CLEAN UP ****
# **** PROCEED WITH CAUTION ****
# **** CLEAN UP ****


In [28]:
# **** PROCEED WITH CAUTION ****

# Once uploaded to IBM COS bucket, use the utilities below to clean up and remove the model directory and its files
# Removing files will free up storage space on this workbench
# First, list the current directories
current_directory_contents = os.listdir('.')
print(f"Contents of current directory: {current_directory_contents}")

Contents of current directory: ['.cache', 'Untitled1.ipynb', 'test_model_download_pipeline.yaml', '.npm', 'test_model_all_download_pipeline.yaml', 'NB-pipeline-dev-helper.ipynb', 'NB-ai-agent-v2-loan-risk-for-rhoai.ipynb', '.ipynb_checkpoints', 'gs_model_pipeline.yaml', 'mcp3-ai-agent-bank-loan.ipynb', 'models--meta-llama--Llama-3.2-3B-Instruct', '.virtual_documents', '.config', 'writing_a_test_file.txt', 'lost+found', 'mcp3-wxai-ai-agent-bank-loan.ipynb', 'Untitled.ipynb', 'gs_hf_aws_pipeline.ipynb', 'file-write-pipeline.yaml', 'full_modeld_pipeline.yaml', '.ipython', '.local', 'NB-gen-rhoai-model-setup-pipeline.ipynb', '.locks', 'full_model_pipeline.yaml', 'my_output_model_path', '.jupyter', 'NB-get-and-upload-model.ipynb']


In [12]:
# Set the local model directory and cache for removal
directory_to_remove=local_models_directory.split(os.sep)[1] #or set this to any other directory 'models--ibm-granite--granite-3.3-8b-instruct'
#directory_to_remove="models--ibm-granite--granite-3.3-8b-instruct"
#directory_to_remove="models--meta-llama--Llama-3.2-3B-Instruct"
cache_directory_to_remove=".cache/huggingface"

path_to_directory_to_remove = os.path.join(current_directory, directory_to_remove)
path_to_cache_directory_to_remove = os.path.join(current_directory, cache_directory_to_remove)

if os.path.exists(path_to_directory_to_remove) and os.path.isdir(path_to_directory_to_remove):
    print(f"*** Model directory to remove: {path_to_directory_to_remove}")
    directory_to_remove_contents = os.listdir(path_to_directory_to_remove)
    print(f"*** Contents of model directory to remove: {directory_to_remove_contents}")
else:
    print("Incorrect directory name or directory does not exist.")
if os.path.exists(cache_directory_to_remove):
    print(f"*** Cache directory to remove: {path_to_cache_directory_to_remove}")
    cache_directory_to_remove_contents = os.listdir('.cache/huggingface')
    print(f"*** Contents of cache directory to remove: {cache_directory_to_remove_contents}")
else:
    print("Cache directory for does not exist.")

*** Model directory to remove: /opt/app-root/src/models--ibm-granite--granite-3.3-8b-instruct
*** Contents of model directory to remove: ['snapshots', 'blobs', 'refs']
*** Cache directory to remove: /opt/app-root/src/.cache/huggingface
*** Contents of cache directory to remove: ['xet']


In [13]:
# NOTE: 
# Running this cell will remove the local model directory and its content files (identified above).
# The files are not needed and can be removed if they were uplaoded to IBM Cloud COS bucket.
# Removing files will free up storage space on this workbench

if os.path.exists(path_to_directory_to_remove) and os.path.isdir(path_to_directory_to_remove):
    shutil.rmtree(path_to_directory_to_remove)
if os.path.exists(path_to_cache_directory_to_remove): 
    shutil.rmtree(path_to_cache_directory_to_remove)

In [4]:
# Confirm the model directory is removed. It should not show up in teh list below.
current_directory_contents = os.listdir('.')
print(f"Contents of current directory: {current_directory_contents}")

Contents of current directory: ['.cache', 'Untitled1.ipynb', 'test_model_download_pipeline.yaml', '.npm', 'test_model_all_download_pipeline.yaml', 'NB-pipeline-dev-helper.ipynb', 'NB-ai-agent-v2-loan-risk-for-rhoai.ipynb', '.ipynb_checkpoints', 'gs_model_pipeline.yaml', 'mcp3-ai-agent-bank-loan.ipynb', 'models--meta-llama--Llama-3.2-3B-Instruct', '.virtual_documents', '.config', 'writing_a_test_file.txt', 'lost+found', 'mcp3-wxai-ai-agent-bank-loan.ipynb', 'Untitled.ipynb', 'gs_hf_aws_pipeline.ipynb', 'file-write-pipeline.yaml', 'full_modeld_pipeline.yaml', '.ipython', '.local', 'NB-gen-rhoai-model-setup-pipeline.ipynb', '.locks', 'full_model_pipeline.yaml', 'my_output_model_path', '.jupyter', 'NB-get-and-upload-model.ipynb']


In [15]:
# Get disk usage statistics again to confirm free space.
total, used, free = shutil.disk_usage(current_directory)

# Convert bytes to a more readable unit (e.g., GB)
total_gb = total / (1024**3)
used_gb = used / (1024**3)
free_gb = free / (1024**3)

current_directory_contents = os.listdir('.')
# Print the results
print(f"Current directory:{current_directory}")
print(f"Disk Usage for {current_directory}:")
print(f"Total: {total_gb:.2f} GB")
print(f"Used: {used_gb:.2f} GB")
print(f"Free: {free_gb:.2f} GB")

Current directory:/opt/app-root/src
Disk Usage for /opt/app-root/src:
Total: 39.20 GB
Used: 0.03 GB
Free: 39.15 GB
